Since you are operating in a Session-Based Architecture (without a central Metastore/Catalog), you cannot simply go to the "Catalog" tab or run standard SELECT * FROM table commands in the SQL Editor. The tables "physically" exist in S3, but the Databricks UI doesn't know they exist.

To query them, you must register them as Temporary Views inside your notebook. This bridges the gap between your Python credentials and SQL syntax.

In [0]:
ACCESS_KEY = dbutils.secrets.get(scope = "ticker", key = "access_key")
SECRET_KEY = dbutils.secrets.get(scope = "ticker", key = "secret_key")
SESSION_TOKEN = dbutils.secrets.get(scope = "ticker", key = "session_key")

temp_ak = dbutils.jobs.taskValues.get(taskKey="Init_Auth", key="temp_ak", debugValue="debug-key")
temp_sk = dbutils.jobs.taskValues.get(taskKey="Init_Auth", key="temp_sk", debugValue="debug-secret")
temp_token = dbutils.jobs.taskValues.get(taskKey="Init_Auth", key="temp_token", debugValue="debug-token")

In [0]:
aws_creds = {
    "fs.s3a.access.key": ACCESS_KEY,
    "fs.s3a.secret.key": SECRET_KEY,
    "fs.s3a.session.token": SESSION_TOKEN,
    "fs.s3a.aws.credentials.provider": "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider"
}

# 2. Define Table Registry (Single Source of Truth)
# Maps the Managed Table Name -> The S3 Source Path
sync_config = [
  
     {
        "table_name": "gold_valid_audit",
        "s3_path": "s3a://mzon-to-databricks-5482/gold/valid",
        "partition_col": "date"
    },
      {
        "table_name": "gold_quarantine_audit",
        "s3_path": "s3a://mzon-to-databricks-5482/gold/invalid",
        "partition_col": "date"
    }
]

# 3. The Optimized Sync Function
def sync_s3_to_managed(table_config):
    table_name = table_config["table_name"]
    path = table_config["s3_path"]
    part_col = table_config.get("partition_col")

    try:
        print(f"🔄 Syncing {table_name}...")
        
        # READ (Securely from S3)
        df = (spark.read.format("delta")
              .options(**aws_creds)
              .load(path))

        # WRITE (To Managed Table)
        # OPTIMIZATION: We add .partitionBy() to ensure the managed table 
        # has the exact same physical structure as S3.
        writer = df.write.mode("overwrite").format("delta")
        
        if part_col:
            writer = writer.partitionBy(part_col)
            
        writer.saveAsTable(table_name)
        
        print(f"Success: {table_name} synced with partitioning on '{part_col}'.")

    except Exception as e:
        print(f"Skipped {table_name}: {str(e)}")

# 4. Execute Loop
for config in sync_config:
    sync_s3_to_managed(config)

In [0]:
%sql
select * from gold_quarantine_audit